<a href="https://colab.research.google.com/github/ferdinandrafols/IA_LLMs/blob/main/aula5_encoder_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Comparando o Modelo Base e o Modelo Fine-Tuned

Agora vamos testar um prompt que não estava nos dados de treino, mas que é do mesmo domínio, e comparar a saída do modelo base com a do modelo fine-tuned para ver o quanto o modelo 'aprendeu' sobre o novo domínio.

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Define necessary variables if previous cells were not run
# (These are typically defined in cells a2667029 and 54f0d9ea)
if 'device' not in locals():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if 'model_id' not in locals():
    model_id = "distilbert/distilgpt2"

if 'tok' not in locals():
    tok = AutoTokenizer.from_pretrained(model_id)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token

# The 'model' variable for fine-tuned inference is expected to be loaded and trained in previous cells.
# If it's not defined, this block will initialize it as a base model to prevent NameError.
# For a true fine-tuned comparison, ensure cells like 54f0d9ea and 99b58af8 are run beforehand.
if 'model' not in locals():
    model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

# Definir um prompt novo, não presente nos dados de treino
new_prompt_text = "Acredito que todos nós devemos construir pontes e não muros."

# Carregar o modelo base novamente para comparação
base_model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

print("Prompt para inferência:", new_prompt_text)

# Inferência com o MODELO BASE
print("\n--- Inferência com o Modelo BASE ---")
base_prompt_tokens = tok(new_prompt_text, return_tensors="pt", truncation=True).input_ids[0,]
base_inputs = base_prompt_tokens.unsqueeze(0).to(device)

with torch.no_grad():
    base_out_ids = base_model.generate(
        base_inputs,
        max_new_tokens=20,
        do_sample=True,
        pad_token_id=tok.eos_token_id,
        temperature=0.7,
        top_k=50,
        top_p=1.0
    )
base_generated_text = tok.decode(base_out_ids[0], skip_special_tokens=True)
print(f"Modelo Base: {base_generated_text}")

# Inferência com o MODELO FINE-TUNED
print("\n--- Inferência com o Modelo FINE-TUNED ---")
fine_tuned_prompt_tokens = tok(new_prompt_text, return_tensors="pt", truncation=True).input_ids[0,]
fine_tuned_inputs = fine_tuned_prompt_tokens.unsqueeze(0).to(device)

with torch.no_grad():
    fine_tuned_out_ids = model.generate(
        fine_tuned_inputs,
        max_new_tokens=20,
        do_sample=True,
        pad_token_id=tok.eos_token_id,
        temperature=0.7,
        top_k=50,
        top_p=1.0
    )
fine_tuned_generated_text = tok.decode(fine_tuned_out_ids[0], skip_special_tokens=True)
print(f"Modelo Fine-Tuned: {fine_tuned_generated_text}")

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilbert/distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Prompt para inferência: Acredito que todos nós devemos construir pontes e não muros.

--- Inferência com o Modelo BASE ---
Modelo Base: Acredito que todos nós devemos construir pontes e não muros. El dos építras (se es como que todo) está. �

--- Inferência com o Modelo FINE-TUNED ---
Modelo Fine-Tuned: Acredito que todos nós devemos construir pontes e não muros. Não muros não muros e och não muros deve


# Aula 4 — Fine-tuning de um Transformer Decoder-only

Nesta prática faremos um **fine-tuning supervisionado (SFT)** em um **decoder-only causal LM** usando o menor modelo *útil* e popular para demos no Hugging Face:

- **Modelo base:** `distilbert/distilgpt2`

Passos:
1. Carregar/definir documentos (toy ou pasta local)
2. Tokenização + *grouping* em blocos (CLM)
3. Fine-tuning com `Trainer`
4. Inferência

In [5]:
# (Opcional) Instalar dependências no ambiente local/Colab
!pip -q install transformers datasets accelerate

In [6]:
import os
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer
)

print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch: 2.9.0+cpu
cuda available: False


## 1) Dataset — Documentos (toy) ou pasta local

### Opção A (didática): documentos toy
### Opção B (real): ler `.txt` de uma pasta local

> Para aula, a Opção A roda rápido e mostra o pipeline completo.


In [7]:
# Opção A: documentos toy
docs = [
    "UFU forma estudantes de IA aplicada para muitas áreas.",
    "Um transformer decoder-only prevê o próximo token com auto-regressão.",
    "Fine-tuning ajusta um modelo pré-treinado para um domínio específico.",
    "Temperatura controla aleatoriedade; top-k e top-p controlam o corte do vocabulário.",
    "A Universidade Federal de Uberlândia é top demais.",
    "A Universidade Federal de Uberlândia é top demais mesmo."
]

ds = Dataset.from_dict({"text": docs}).train_test_split(test_size=0.1, seed=42)
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 5
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1
    })
})

In [8]:
from datasets import Dataset
from glob import glob
# The user provided a specific file to use as the dataset.
# We will read this single file into our documents.
file_path = "/content/500 Frases Mais Famosas do Obama (1).txt"

try:
    with open(file_path, "r", encoding="utf-8") as f:
        # Split the content by lines to create multiple documents
        docs = [line.strip() for line in f.readlines() if line.strip()]

    if not docs:
        raise ValueError("No content found in the file or all lines were empty.")

    # Adjust test_size if there are very few documents
    test_size = 0.1
    if len(docs) < 2:
        # If only one document, use it for both train and test (or skip split if not feasible)
        print("Warning: Only one document found. Skipping train_test_split.")
        ds = Dataset.from_dict({"text": docs})
    else:
        ds = Dataset.from_dict({"text": docs}).train_test_split(test_size=test_size, seed=42)

    print("Dataset loaded from:", file_path)
    print(ds)

except FileNotFoundError:
    print(f"Error: The file {file_path} was not found. Please ensure the file exists.")
    docs = []
    ds = Dataset.from_dict({"text": []})
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    docs = []
    ds = Dataset.from_dict({"text": []})

Error: The file /content/500 Frases Mais Famosas do Obama (1).txt was not found. Please ensure the file exists.


## 2) Tokenizer + Modelo base

Usaremos o tokenizer do modelo base e definiremos `pad_token`, pois GPT-2 não tem por padrão.


In [9]:
model_id = "distilbert/distilgpt2"

tok = AutoTokenizer.from_pretrained(model_id)

# GPT-2 não define pad_token por padrão; necessário para batches com padding
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

print("Vocab size:", len(tok))
print("Model loaded:", model_id)

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilbert/distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Vocab size: 50257
Model loaded: distilbert/distilgpt2


## 3) Tokenização + agrupamento em blocos (Causal LM)

Em *Causal Language Modeling*, a label é o próprio `input_ids` deslocado internamente pelo modelo.
Uma forma padrão é concatenar tudo e quebrar em blocos fixos (`block_size`).


In [10]:
block_size = 64  # pequeno para rodar rápido

def tokenize_fn(batch):
    return tok(batch["text"], truncation=True)

tok_ds = ds.map(tokenize_fn, batched=False, remove_columns=["text"])

def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated["input_ids"])
    print('total_len = ', total_length)
    total_length = (total_length // block_size) * block_size
    print('total_len = ', total_length)
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_ds = tok_ds.map(group_texts, batched=True)
collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

## 4) Fine-tuning (SFT) com `Trainer`

> Com o seguinte código, abstraímos a parte do treino e executar 10 épocas para o modelo aprender o conteúdo do novo corpus.


In [11]:
args = TrainingArguments(
    output_dir="./ft_tiny_gpt2",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    learning_rate=5e-4,
    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="no",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=lm_ds["train"],
    eval_dataset=lm_ds["test"],
    data_collator=collator
)

trainer.train()

ValueError: Column 'train' doesn't exist.

## 5) Inferência depos do finetune



In [ ]:
prompt=tok("UFU ", return_tensors = "pt", truncation = True).input_ids[0,]
inputs = prompt.unsqueeze(0).to(device)
with torch.no_grad():
    out_ids = model.generate(
        inputs,
        max_new_tokens=10,
        do_sample=True,
        pad_token_id=tok.eos_token_id,
        temperature= 0.7,
        top_k = 50,
        top_p = 1.0
    )
print(f"Inference: {tok.decode(out_ids[0])}")

In [ ]:
print("\n--- Inferência em Documentos de Treino ---")
train_docs = ds['train']['text']

for i, doc_text in enumerate(train_docs):
    initial_tokens = tok(doc_text, return_tensors="pt", truncation=True).input_ids[0, :10]
    inputs = initial_tokens.unsqueeze(0).to(device) # unsqueeze to add batch dimension

    with torch.no_grad():
        out_ids = model.generate(
            inputs,
            max_new_tokens=10,
            do_sample=True,
            pad_token_id=tok.eos_token_id,
            temperature= 0.7,
            top_k = 50,
            top_p = 1.0
        )

    # Decode the sequence
    prompt_text = tok.decode(initial_tokens, skip_special_tokens=True)
    generated_text = tok.decode(out_ids[0], skip_special_tokens=True)
    print(f"doc{i}: {prompt_text}*{generated_text[len(prompt_text):]}*")


## Glossário de Conceitos e Técnicas

Aqui estão os principais conceitos e técnicas abordados nesta prática de fine-tuning:

*   **Fine-tuning Supervisionado (SFT)**: Um processo de ajuste de um modelo pré-treinado em um conjunto de dados específico (com rótulos) para adaptá-lo a uma tarefa ou domínio particular. Neste caso, é usado para o modelo aprender o estilo e conteúdo do novo corpus.

*   **Decoder-only Causal LM (Causal Language Model)**: Um tipo de arquitetura de modelo de linguagem que prevê o próximo token na sequência com base apenas nos tokens anteriores. É 'causal' porque a previsão de um token não pode depender de tokens futuros. É a arquitetura comum para modelos como GPT (Generative Pre-trained Transformer).

*   **Modelo Base**: Refere-se ao modelo pré-treinado (`distilbert/distilgpt2` neste caso) que serve como ponto de partida antes de qualquer fine-tuning.

*   **Tokenização**: O processo de dividir texto bruto em unidades menores chamadas 'tokens'. Estes tokens podem ser palavras, subpalavras ou caracteres, dependendo do tokenizer. Essencial para converter texto em um formato que o modelo pode processar.

*   **Grouping em Blocos (CLM)**: Uma técnica comum em Causal Language Modeling onde o texto tokenizado é concatenado e então dividido em blocos de tamanho fixo (`block_size`). Isso permite que o modelo processe sequências mais longas de texto e cria mais exemplos de treinamento para o modelo aprender as dependências de longo alcance.

*   **Hugging Face `Trainer`**: Uma classe utilitária da biblioteca Hugging Face Transformers que simplifica e padroniza o processo de treinamento e avaliação de modelos de linguagem, abstraindo muitos detalhes de baixo nível.

*   **`AutoTokenizer`**: Uma classe da biblioteca Hugging Face Transformers que permite carregar automaticamente o tokenizer apropriado para um modelo pré-treinado específico (ex: `AutoTokenizer.from_pretrained('distilbert/distilgpt2')`).

*   **`AutoModelForCausalLM`**: Uma classe da biblioteca Hugging Face Transformers que permite carregar automaticamente o modelo de linguagem causal apropriado para um modelo pré-treinado específico (ex: `AutoModelForCausalLM.from_pretrained('distilbert/distilgpt2')`).

*   **`DataCollatorForLanguageModeling`**: Uma ferramenta do Hugging Face que prepara os lotes (batches) de dados para o treinamento de modelos de linguagem, lidando com o preenchimento (padding) e a geração de rótulos (labels) conforme necessário para o CLM (onde os rótulos são os próprios `input_ids` deslocados).

*   **Temperatura (durante inferência)**: Um parâmetro que controla a aleatoriedade das previsões do modelo. Temperaturas mais altas (`>1.0`) aumentam a aleatoriedade (criatividade), enquanto temperaturas mais baixas (`<1.0`) tornam a saída mais determinística e focada.

*   **Top-k (durante inferência)**: Uma estratégia de amostragem na geração de texto onde o modelo considera apenas os `k` tokens com as maiores probabilidades para a próxima previsão, antes de amostrar um deles.

*   **Top-p (durante inferência) / Nucleus Sampling**: Outra estratégia de amostragem que seleciona o menor conjunto de tokens cuja soma das probabilidades excede `p`, e então amostra um token desse conjunto. Isso permite um controle mais dinâmico sobre a variedade da geração.

*   **`pad_token`**: Um token especial usado para preencher sequências mais curtas em um lote, para que todas as sequências tenham o mesmo comprimento. Essencial para processamento em batch eficiente por GPUs.

*   **`eos_token` (End-of-Sequence Token)**: Um token especial que indica o fim de uma sequência ou frase. Em muitos modelos, é usado para sinalizar ao modelo que ele deve parar de gerar texto.

# Exercício prático

Escolha um prompt que não estava nos dados de treino, mas que seja do mesmo domínio. Faça comparações. Documente a saída do modelo base (sem treino) e do modelo ajustado. Verifique se o modelo "aprendeu" termos técnicos ou o vocabulário específico que você forneceu nos arquivos .txt.

# Task
Continue fine-tuning the language model by re-loading the Obama quotes from "/content/500 Frases Mais Famosas do Obama (1).txt", then tokenize and group the data for Causal Language Modeling, fine-tune the model with the prepared dataset, and finally compare the inference output of the base and fine-tuned models to observe the learning.

## Load Obama Quotes Dataset

### Subtask:
Re-execute the cell `fe82c526` to load the text from '/content/500 Frases Mais Famosas do Obama (1).txt' into the `ds` dataset. This step will ensure the dataset is correctly populated for fine-tuning.


**Reasoning**:
The subtask explicitly requests to re-execute cell `fe82c526` to load the dataset. This command directly fulfills that requirement.



In [12]:
from datasets import Dataset
from glob import glob
# The user provided a specific file to use as the dataset.
# We will read this single file into our documents.
file_path = "/content/500 Frases Mais Famosas do Obama (1).txt"

try:
    with open(file_path, "r", encoding="utf-8") as f:
        # Split the content by lines to create multiple documents
        docs = [line.strip() for line in f.readlines() if line.strip()]

    if not docs:
        raise ValueError("No content found in the file or all lines were empty.")

    # Adjust test_size if there are very few documents
    test_size = 0.1
    if len(docs) < 2:
        # If only one document, use it for both train and test (or skip split if not feasible)
        print("Warning: Only one document found. Skipping train_test_split.")
        ds = Dataset.from_dict({"text": docs})
    else:
        ds = Dataset.from_dict({"text": docs}).train_test_split(test_size=test_size, seed=42)

    print("Dataset loaded from:", file_path)
    print(ds)

except FileNotFoundError:
    print(f"Error: The file {file_path} was not found. Please ensure the file exists.")
    docs = []
    ds = Dataset.from_dict({"text": []})
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    docs = []
    ds = Dataset.from_dict({"text": []})

Dataset loaded from: /content/500 Frases Mais Famosas do Obama (1).txt
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 270
    })
    test: Dataset({
        features: ['text'],
        num_rows: 30
    })
})


**Reasoning**:
The dataset `ds` has been successfully loaded with the Obama quotes. The next step in the fine-tuning process is to tokenize the text data and group it into fixed-size blocks for Causal Language Modeling, as specified by the overall task.



In [13]:
block_size = 64  # pequeno para rodar rápido

def tokenize_fn(batch):
    return tok(batch["text"], truncation=True)

tok_ds = ds.map(tokenize_fn, batched=False, remove_columns=["text"])

def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated["input_ids"])
    print('total_len = ', total_length)
    total_length = (total_length // block_size) * block_size
    print('total_len = ', total_length)
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_ds = tok_ds.map(group_texts, batched=True)
collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

total_len =  3063
total_len =  3008


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

total_len =  348
total_len =  320


**Reasoning**:
The data has been tokenized and grouped for Causal Language Modeling. The next step is to fine-tune the model using the prepared `lm_ds` dataset and the Hugging Face `Trainer` as described in the overall task.



In [14]:
args = TrainingArguments(
    output_dir="./ft_tiny_gpt2",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    learning_rate=5e-4,
    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="no",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=lm_ds["train"],
    eval_dataset=lm_ds["test"],
    data_collator=collator
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.020652,2.920737
2,1.646516,3.291842
3,1.260301,3.431842
4,1.116676,3.542227
5,0.791227,3.943805
6,0.584562,4.408114
7,0.439681,4.672380
8,0.245149,4.823079
9,0.281424,5.294633


KeyboardInterrupt: 

# Task
Continue fine-tuning the language model by re-loading the Obama quotes from "/content/500 Frases Mais Famosas do Obama (1).txt", then tokenize and group the data for Causal Language Modeling, fine-tune the model with the prepared dataset, and finally compare the inference output of the base and fine-tuned models to observe the learning.

## Load Encoder-only Model

### Subtask:
Load a pre-trained Encoder-only model (e.g., from `sentence-transformers`) and its tokenizer, specifically designed for generating embeddings.


**Reasoning**:
The first instruction is to install the `sentence-transformers` library, which is a prerequisite for loading the model.



In [15]:
get_ipython().system('pip install -q sentence-transformers')

**Reasoning**:
Following the instructions, the `SentenceTransformer` class needs to be imported, and then a pre-trained model must be instantiated to load the Encoder-only model.



In [16]:
from sentence_transformers import SentenceTransformer

# Instantiate a SentenceTransformer model
# 'paraphrase-multilingual-MiniLM-L12-v2' is chosen as it is suitable for multilingual text,
# which might be relevant given the Obama quotes are in Portuguese.
encoder_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

print("Encoder-only model loaded successfully.")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/526 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Encoder-only model loaded successfully.


**Reasoning**:
The previous fine-tuning process was interrupted by a `KeyboardInterrupt`, meaning the model was not fully fine-tuned on the Obama quotes. To proceed with the overall task of comparing base and fine-tuned models, the fine-tuning step must be completed successfully.



In [ ]:
args = TrainingArguments(
    output_dir="./ft_tiny_gpt2",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    learning_rate=5e-4,
    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="no",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=lm_ds["train"],
    eval_dataset=lm_ds["test"],
    data_collator=collator
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,0.487643,4.560915
2,0.449968,4.681026
3,0.375216,5.388311
4,0.207169,5.586585
